<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/Overview_of_Colaboratory_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tf-agents[reverb]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np
import pandas as pd
import reverb

import tf_agents as ta
from tf_agents.networks import network
import tensorflow_probability as tfp
from tf_agents.policies import py_tf_eager_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tf_agents.drivers import py_driver
from tf_agents.utils import common
import tensorflow as tf
from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import sequential
from tf_agents.specs import tensor_spec
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.policies import random_tf_policy
from tf_agents.drivers import dynamic_step_driver

RuntimeError: ignored

In [5]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 2  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

In [6]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data.csv')
df.pop("index")
del df[df. columns[0]]
df = df.astype('float64')
min = df.min()
obsShape = len(df. columns)

dfarray = df.to_numpy()
dflist = dfarray.tolist()

length = len(dflist)

print(dflist[1][3])
print(df)
print(np.asarray(dflist).shape)
#print(df.dtypes)

-0.0001999999999999
       Direction  Bottom_Point  Top_Point  Relative_Change     Size   day  \
0            0.0      0.000000   0.666667         -0.00080  0.00120   2.0   
1            0.0      0.444444   0.666667         -0.00020  0.00090   2.0   
2            1.0      0.230769   0.846154          0.00080  0.00130   2.0   
3            0.0      0.888889   0.888889          0.00000  0.00090   2.0   
4            1.0      0.400000   0.700000          0.00030  0.00100   2.0   
...          ...           ...        ...              ...      ...   ...   
93075        1.0      0.575000   0.962500          0.00093  0.00240  29.0   
93076        1.0      0.457317   0.682927          0.00037  0.00164  29.0   
93077        0.0      0.422680   0.824742         -0.00039  0.00097  29.0   
93078        1.0      0.634146   0.780488          0.00006  0.00041  29.0   
93079        1.0      0.515152   0.969697          0.00015  0.00033  29.0   

       month  weekday  hour_of_day  
0        5.0      

In [7]:
class TradingEnv(py_environment.PyEnvironment):

  
  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(9,), dtype=np.float32, name='observation')
    self.counter = 0
    self.reward = 0
    self._state = dflist[0]
    self._episode_ended = False
    self._reward_spec = array_spec.BoundedArraySpec(
    shape=(1,), dtype=np.float32, minimum=-100.0, maximum=100.0, name='reward')

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = dflist[0]
    self._episode_ended = False
    self.reward = 0
    self.counter = 0
    return ts.restart(np.array([self._state], dtype=np.float32))

  def _step(self, action):

    self.counter += 1

    self._state = dflist[self.counter]

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Buy Sell reward.
    if action == 1:
      self.reward += dflist[self.counter][3] 
    elif action == 0:
      self.reward -= dflist[self.counter][3]
    else:
      raise ValueError('`action` should be 0 or 1.')


    if self.counter == 1000:
      self._episode_ended = True


    if self._episode_ended:
      return ts.termination(np.array([self._state], dtype=np.float32), reward=self.reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.float32), reward=self.reward, discount=1)

In [8]:
custom_py_env = CustomEnvironment()
custom_tf_env = tf_py_environment.TFPyEnvironment(custom_py_env)

In [9]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

def dense_layer(num_units):
    return tf.keras.layers.Dense(
        num_units,
        activation=tf.keras.activations.relu,
        kernel_initializer=tf.keras.initializers.VarianceScaling(
            scale=2.0, mode='fan_in', distribution='truncated_normal'))

dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = tf.keras.Sequential(dense_layers + [Flatten(), q_values_layer])

In [10]:
class QNetwork(network.Network):

    def __init__(self, input_shape, num_actions, fc_layer_params=(100, 50), name='QNetwork'):
        super(QNetwork, self).__init__(input_tensor_spec=None, state_spec=(), name=name)
        self._num_actions = num_actions
        self._fc_layers = [dense_layer(num_units) for num_units in fc_layer_params]
        self._q_values_layer = tf.keras.layers.Dense(
            num_actions,
            activation=None,
            kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.03, maxval=0.03),
            bias_initializer=tf.keras.initializers.Constant(-0.2))

    def call(self, inputs, step_type=None, network_state=()):
        x = tf.cast(inputs, tf.float32)
        for layer in self._fc_layers:
            x = layer(x)
        q_values = self._q_values_layer(x)
        q_values = tf.reshape(q_values, [-1, self._num_actions])  # Add this line to reshape the output tensor
        return q_values, network_state


In [11]:
q_net = QNetwork(env.observation_spec().shape, num_actions)

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
train_step_counter = tf.Variable(0)
agent = dqn_agent.DqnAgent(
    env.time_step_spec(),
    env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)
agent.initialize()

In [12]:
replay_buffer = ta.replay_buffers.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=1000,
    max_length=100000)


In [13]:
# Create a random policy for initial data collection
random_policy = random_tf_policy.RandomTFPolicy(
    env.time_step_spec(),
    env.action_spec())

# Create a driver to collect experience from the environment
collect_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    random_policy,
    observers=[replay_buffer.add_batch],
    num_steps=1000)  # Collect 1000 steps

# Collect initial experience
collect_driver.run()


ValueError: ignored